##### Imports

In [ ]:
from operator import itemgetter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import aim

import plotly.express as px

from skimage.draw import disk

from medpy.metric.binary import dc

from monai.metrics import DiceMetric, HausdorffDistanceMetric, compute_meandice
from monai.transforms import AsDiscrete, EnsureType, Compose
from monai.data import decollate_batch
from monai.losses import DiceLoss
from monai.networks import one_hot

import plotly.express as px

import kornia.augmentation as K

import torch
from torch import nn
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DataLoader, TensorDataset

from kedro.extras.datasets.pickle import PickleDataSet

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../src'))

from tagseg.models.segmenter import Net
from tagseg.models.trainer import Trainer
from tagseg.metrics.shape import ShapeDistLoss
from tagseg.pipelines.data_splitting.nodes import split_data

##### Fetch data

In [ ]:
dataset = PickleDataSet(filepath='../data/05_model_input/model_input.pt').load()

In [ ]:
len(dataset)

In [ ]:
laplace_kernel = nn.Conv2d(1, 1, (3, 3), bias=False)
laplace_kernel.weight = torch.nn.Parameter(torch.tensor([[[[0, 1, 0], [1, -4, 1], [0, 1, 0]]]]).double())

In [ ]:
storage = []

for image, label in dataset:

    information = dict(
        mean=image.mean().item(),
        std=image.std().item(),
        label_pixels=label.sum().item(),
        variance_lap=laplace_kernel(image.unsqueeze(0)).var().item()
    )

    storage.append(information)

df = pd.DataFrame(storage)

In [ ]:
df.head()

In [ ]:
px.histogram(df, x=['mean', 'std'])

In [ ]:
px.histogram(df, 'label_pixels')

In [ ]:
px.histogram(df, 'variance_lap')